# Example of querying

In [1]:
import folium
import requests
import json

## Admin 1 level queries

In [ ]:
iso = 'DEU'

In [ ]:
query = f"""with gadm28 as (select the_geom_webmercator from  gadm28_adm1 where iso='{iso}')
           SELECT gadm28.the_geom_webmercator, count(synthesys.home_insti)
           FROM  sanitized_new_data as synthesys, gadm28
           where ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
           group by gadm28.the_geom_webmercator"""

style = """#layer {
  polygon-fill: ramp([count], colorbrewer(Reds), jenks());
  polygon-opacity: 0.9;
}"""

In [ ]:
account = 'synthesys'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}


r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm)
map_osm

## Country-wide queries

In [5]:
# query="""with gadm28 as (select st_buffer(st_union(the_geom_webmercator), 0.001) as the_geom_webmercator
#          from  gadm28_adm1),
#             grouped as (SELECT gadm28.the_geom_webmercator, count(synthesys.home_insti)
#          FROM sanitized_new_data as synthesys, gadm28
#          where ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
#          and discipline like  'Earth Sciences & Environment' 
#          group by gadm28.the_geom_webmercator)
#           select * from grouped where count >1 """ 

query = """WITH gadm28 AS (SELECT the_geom_webmercator, iso2 as iso FROM  gadm28_countries)
 SELECT gadm28.the_geom_webmercator, gadm28.iso, COUNT(synthesys.home_insti) AS count
 FROM  sanitized_data AS synthesys, gadm28
 WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 and synthesys.discipline like 'Earth Sciences & Environment'
 GROUP BY gadm28.the_geom_webmercator, gadm28.iso"""


style = """#layer {polygon-fill: ramp([count], colorbrewer(Reds), jenks());
                   polygon-opacity: 0.9;}"""

In [6]:
query

"WITH gadm28 AS (SELECT the_geom_webmercator, iso2 as iso FROM  gadm28_countries)\n SELECT gadm28.the_geom_webmercator, gadm28.iso, COUNT(synthesys.home_insti) AS count\n FROM  sanitized_data AS synthesys, gadm28\n WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)\n and synthesys.discipline like 'Earth Sciences & Environment'\n GROUP BY gadm28.the_geom_webmercator, gadm28.iso"

In [7]:
account = 'synthesys'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}

r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm)
map_osm

{'layergroupid': '1a34bd6fc460b4e56eba432764b6fb04:1501074438210', 'metadata': {'layers': [{'type': 'mapnik', 'id': 'layer0', 'meta': {'cartocss': '#layer {polygon-fill: #fee5d9;[ count > 22 ] {polygon-fill: #fcae91;}[ count > 66 ] {polygon-fill: #fb6a4a;}[ count > 204 ] {polygon-fill: #de2d26;}[ count > 320 ] {polygon-fill: #a50f15;}\n                   polygon-opacity: 0.9;}', 'stats': {'estimatedFeatureCount': -1}, 'cartocss_meta': {'rules': [{'selector': '#layer', 'prop': 'polygon-fill', 'column': 'count', 'mapping': '>', 'buckets': [{'filter': {'type': 'range', 'start': 1, 'end': 22}, 'value': '#fee5d9'}, {'filter': {'type': 'range', 'start': 22, 'end': 66}, 'value': '#fcae91'}, {'filter': {'type': 'range', 'start': 66, 'end': 204}, 'value': '#fb6a4a'}, {'filter': {'type': 'range', 'start': 204, 'end': 320}, 'value': '#de2d26'}, {'filter': {'type': 'range', 'start': 320, 'end': 475}, 'value': '#a50f15'}], 'stats': {'filter_avg': 80.9}}]}}}], 'dataviews': {}, 'analyses': []}, 'cdn_

## Admin-level 1 across europe


In [ ]:
query = """WITH gadm28 AS (SELECT the_geom_webmercator, iso FROM  gadm28_adm1)
 SELECT gadm28.the_geom_webmercator, gadm28.iso, COUNT(synthesys.home_insti) AS count
 FROM  sanitized_new_data AS synthesys, gadm28
 WHERE ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 GROUP BY gadm28.the_geom_webmercator, gadm28.iso"""




style = """#layer {polygon-fill: ramp([count], colorbrewer(Reds), jenks());
                   polygon-opacity: 0.9;}"""

In [ ]:
account = 'synthesys'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}


r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm)
map_osm

##  Need to create examples of querying the SQL API

Used for building the widgets

In [ ]:
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
sql={"q":"SELECT iso2, name_engli FROM gadm28_countries LIMIT 5"}


r = requests.get(urlCarto, params=sql) # data=json.dumps(body)


In [ ]:
r.json()

### number of men and women with filters

* Men/women (all countries)
* Men/women by country
* Men/women by discipline
* Men/women by Synthysis call

In [ ]:
# Men/women: no filter
sql = {"q":"SELECT gender, count(*) FROM sanitized_new_data group by gender"}
r = requests.get(urlCarto, params=sql) 
r.json()

In [ ]:
# Men/women: filter by country
sql = {"q":"WITH gadm28 SELECT gender, count(*) FROM sanitized_new_data group by gender"}
r = requests.get(urlCarto, params=sql) 
r.json()